## Подготовка исходных данных для обучения модели, которая будет использоваться для рекомендации постов в социальной сети.
#### - Извлечение данных с серверов PostgreSQL
#### - Обработка и выделение дополнительных признаков для последующего обучения модели.
#### - Загрузка признаков на сервер для дальнейшего использования.

In [29]:
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn.decomposition import PCA

In [11]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [12]:
user_info = pd.read_sql("""SELECT * FROM public.user_data""", con=engine)


In [13]:
print(user_info.shape)
user_info.head(2)

(163205, 8)


,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads


In [16]:
posts_info = pd.read_sql("""SELECT * FROM public.post_text_df""", con=engine)

In [17]:
print(posts_info.shape)
posts_info.head(2)

(7023, 3)


,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business


### Обработка таблицы user_info

#### Как мы помним, у пользователей есть явные предпочтения в отношении определенных тематик, которые они чаще просматривают и оценивают положительно. Для каждого пользователя мы извлечем его предпочтения из базы данных. Чем выше значение, тем больше интерес пользователь проявляет к данной тематике.

In [3]:
sql_text = "SELECT public.feed_data.user_id, public.post_text_df.topic, avg(public.feed_data.target) \
            FROM public.feed_data \
            JOIN public.post_text_df ON public.feed_data.post_id=public.post_text_df.post_id \
            WHERE public.feed_data.action='view' \
            GROUP BY public.feed_data.user_id, public.post_text_df.topic;"

In [2]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [4]:
user_like = batch_load_sql(sql_text)

In [5]:
user_like

,user_id,topic,avg
0,200,business,0.071429
1,200,covid,0.096774
2,200,entertainment,0.285714
3,200,movie,0.170455
4,200,politics,0.068966
...,...,...,...
1142350,168552,entertainment,0.000000
1142351,168552,movie,0.039216
1142352,168552,politics,0.000000
1142353,168552,sport,0.185185


In [6]:
user_like = user_like.pivot_table('avg', index='user_id', columns='topic')
user_like

topic,business,covid,entertainment,movie,politics,sport,tech
user_id,,,,,,,
200,0.071429,0.096774,0.285714,0.170455,0.068966,0.109375,0.153846
201,0.103448,0.063953,0.121212,0.122677,0.013889,0.023256,0.137931
202,0.052632,0.139241,0.037037,0.126812,0.254237,0.168831,0.000000
203,0.157895,0.183099,0.181818,0.182482,0.200000,0.224490,0.222222
204,0.125000,0.258065,0.200000,0.140625,0.100000,0.117647,0.333333
...,...,...,...,...,...,...,...
168548,0.083333,0.072289,0.105263,0.047945,0.025641,0.000000,0.250000
168549,0.000000,0.074627,0.000000,0.063158,0.225806,0.160000,0.000000
168550,0.034483,0.117647,0.086957,0.135802,0.000000,0.030303,0.142857


In [69]:
user_like.columns.name = None
user_like = user_like.reset_index()

In [7]:
user_like.isna().sum()

topic
business          6
covid             0
entertainment    20
movie             0
politics          0
sport             0
tech             54
dtype: int64

In [8]:
# Некоторые пользователи не просматривали определенные категории. Заполним nan нулём
user_like.fillna(value=0, inplace=True)

In [19]:
# Объединим полученную таблицу с таблицей признаков пользователей
user_info_data = user_info.merge(user_like, left_on='user_id', right_on='user_id')

In [20]:
user_info_data

,user_id,gender,age,country,city,exp_group,os,source,business,covid,entertainment,movie,politics,sport,tech
0,200,1,34,Russia,Degtyarsk,3,Android,ads,0.071429,0.096774,0.285714,0.170455,0.068966,0.109375,0.153846
1,201,0,37,Russia,Abakan,0,Android,ads,0.103448,0.063953,0.121212,0.122677,0.013889,0.023256,0.137931
2,202,1,17,Russia,Smolensk,4,Android,ads,0.052632,0.139241,0.037037,0.126812,0.254237,0.168831,0.000000
3,203,0,18,Russia,Moscow,1,iOS,ads,0.157895,0.183099,0.181818,0.182482,0.200000,0.224490,0.222222
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads,0.125000,0.258065,0.200000,0.140625,0.100000,0.117647,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic,0.083333,0.072289,0.105263,0.047945,0.025641,0.000000,0.250000
163201,168549,0,18,Russia,Tula,2,Android,organic,0.000000,0.074627,0.000000,0.063158,0.225806,0.160000,0.000000
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic,0.034483,0.117647,0.086957,0.135802,0.000000,0.030303,0.142857
163203,168551,0,38,Russia,Moscow,3,iOS,organic,0.047619,0.131148,0.176471,0.110000,0.026316,0.062500,0.066667


In [21]:
# Версия операционной системы и признак откуда пришел пользователь с органического трафика или с рекламы не являются важными для нашей модели, удалим их.
user_info_data=user_info_data.drop(['os','source'], axis=1)
user_info_data

,user_id,gender,age,country,city,exp_group,business,covid,entertainment,movie,politics,sport,tech
0,200,1,34,Russia,Degtyarsk,3,0.071429,0.096774,0.285714,0.170455,0.068966,0.109375,0.153846
1,201,0,37,Russia,Abakan,0,0.103448,0.063953,0.121212,0.122677,0.013889,0.023256,0.137931
2,202,1,17,Russia,Smolensk,4,0.052632,0.139241,0.037037,0.126812,0.254237,0.168831,0.000000
3,203,0,18,Russia,Moscow,1,0.157895,0.183099,0.181818,0.182482,0.200000,0.224490,0.222222
4,204,0,36,Russia,Anzhero-Sudzhensk,3,0.125000,0.258065,0.200000,0.140625,0.100000,0.117647,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,0.083333,0.072289,0.105263,0.047945,0.025641,0.000000,0.250000
163201,168549,0,18,Russia,Tula,2,0.000000,0.074627,0.000000,0.063158,0.225806,0.160000,0.000000
163202,168550,1,41,Russia,Yekaterinburg,4,0.034483,0.117647,0.086957,0.135802,0.000000,0.030303,0.142857
163203,168551,0,38,Russia,Moscow,3,0.047619,0.131148,0.176471,0.110000,0.026316,0.062500,0.066667


#### Загрузим признаки для всех юзеров на сервер

In [28]:
user_info_data.to_sql('ryagrig_users_lesson_22', con=engine, if_exists='replace')

205

### Обработка таблицы post_info
#### Для первой модели использем TF-IDF. Идея метода TF-IDF заключается в том, чтобы придавать большое значение любому термину, который часто встречается в конкретном посте, но не во многих постах нашей таблицы.

In [22]:
tfidf = TfidfVectorizer(stop_words='english')
start_time = time.time()
tfidf_matrix = tfidf.fit_transform(posts_info['text'])

# вычисление среднего значения TF-IDF для каждого объекта
tfidf_avg = tfidf_matrix.mean(axis=1)

# замена старой колонки с описанием на новую, содержащую среднее значение TF-IDF для каждого объекта
posts_info['text_TFIDF'] = tfidf_avg
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Time elapsed: {elapsed_time} seconds')

Time elapsed: 1.5737895965576172 seconds


In [23]:
# Преобразованные данные в виде матрицы TF-IDF, где каждая строка соответствует одному посту, а каждый столбец соответствует весу TF-IDF для определенного слова в посте.
tfidf_data = tfidf.fit_transform(posts_info['text']).toarray() 
tfidf_data

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.13219051, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05039818, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [24]:
tfidf_data = pd.DataFrame(tfidf_data, index=posts_info.post_id, columns=tfidf.get_feature_names())

C:\Users\ocenk\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [25]:
tfidf_data

,00,000,0001,000bn,000m,000s,000th,001,001and,001st,...,𝓫𝓮,𝓫𝓮𝓽𝓽𝓮𝓻,𝓬𝓸𝓾𝓻𝓽𝓼,𝓱𝓮𝓪𝓻𝓲𝓷𝓰,𝓶𝓪𝔂,𝓹𝓱𝔂𝓼𝓲𝓬𝓪𝓵,𝓼𝓸𝓸𝓷𝓮𝓻,𝓼𝓾𝓫𝓸𝓻𝓭𝓲𝓷𝓪𝓽𝓮,𝓽𝓱𝓮,𝓽𝓸
post_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.132191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.050398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7317,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Добавим три новых столбца: 'TotalTfIdf', 'MaxTfIdf' и 'MeanTfIdf', содержащих, соответственно, сумму, максимальное и среднее значение TF-IDF для каждого поста.

In [26]:
posts_info['TotalTfIdf'] = tfidf_data.sum(axis=1).reset_index()[0]
posts_info['MaxTfIdf'] = tfidf_data.max(axis=1).reset_index()[0]
posts_info['MeanTfIdf'] = tfidf_data.mean(axis=1).reset_index()[0]
posts_info

,post_id,text,topic,text_TFIDF,TotalTfIdf,MaxTfIdf,MeanTfIdf
0,1,UK economy facing major risks\n\nThe UK manufa...,business,0.000179,9.198863,0.499797,0.000179
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,0.000239,12.305151,0.306730,0.000239
2,3,Asian quake hits European shares\n\nShares in ...,business,0.000256,13.203359,0.275247,0.000256
3,4,India power shares jump on debut\n\nShares in ...,business,0.000137,7.064717,0.551670,0.000137
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,0.000131,6.772273,0.424172,0.000131
...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,0.000129,6.662949,0.260875,0.000129
7019,7316,I give this movie 2 stars purely because of it...,movie,0.000126,6.475045,0.391273,0.000126
7020,7317,I cant believe this film was allowed to be mad...,movie,0.000110,5.654622,0.560737,0.000110
7021,7318,The version I saw of this film was the Blockbu...,movie,0.000133,6.863538,0.231077,0.000133


#### Для понижения размерности признаков используем метод главных компонент (PCA). Главная цель найти новые некоррелированные переменные, называемые главными компонентами, которые объясняют максимальное количество дисперсии в исходных данных.

In [27]:
# Центруем значения tfidf
centered = tfidf_data - tfidf_data.mean()

In [30]:
# Понижаем размерность признаков.
pca = PCA(n_components=10)
pca_decomp = pca.fit_transform(centered)

In [31]:
pca_decomp

array([[-0.07970145,  0.1514185 , -0.02513133, ...,  0.10645023,
        -0.11519921,  0.01244493],
       [-0.0736847 ,  0.17274072,  0.04968189, ...,  0.01330782,
        -0.00283431,  0.00358171],
       [-0.05730744,  0.09749233, -0.0249338 , ...,  0.02628666,
        -0.00608694, -0.00181695],
       ...,
       [ 0.07113059, -0.02424187, -0.09168134, ...,  0.02019619,
         0.00532001, -0.00248169],
       [ 0.06408179, -0.00573782, -0.08470069, ..., -0.00554318,
        -0.00386489,  0.00270863],
       [ 0.03338865, -0.0180011 , -0.02314369, ..., -0.03938031,
         0.01127111, -0.00141638]])

####  PCA помог удалить шумовые компоненты из данных и выделить наиболее значимые аспекты в данных, что может быть полезно для повышения качества классификации.Понизив размерность применим KNN, который используется на основе близости объектов в пространстве признаков. Это поможет работать более эффективно и быстро.

In [32]:
n_clusters = 5

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)

# Добавляем признак метка кластера, для каждого поста.
posts_info['TextCluster'] = kmeans.labels_

dists_columns = [f"DistanceTo{ith}thCluster" for ith in range(1, n_clusters + 1)]

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster
0,0.444728,0.460215,0.410973,0.404730,0.327258
1,0.316478,0.201295,0.270981,0.254154,0.148975
2,0.331828,0.392673,0.293103,0.279684,0.197841
3,0.289548,0.339505,0.227644,0.221550,0.121759
4,0.247473,0.309660,0.177376,0.164386,0.081970


In [33]:
posts_info= pd.concat((posts_info,dists_df), axis=1)
posts_info.head()

,post_id,text,topic,text_TFIDF,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,0.000179,9.198863,0.499797,0.000179,4,0.444728,0.460215,0.410973,0.404730,0.327258
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,0.000239,12.305151,0.306730,0.000239,4,0.316478,0.201295,0.270981,0.254154,0.148975
2,3,Asian quake hits European shares\n\nShares in ...,business,0.000256,13.203359,0.275247,0.000256,4,0.331828,0.392673,0.293103,0.279684,0.197841
3,4,India power shares jump on debut\n\nShares in ...,business,0.000137,7.064717,0.551670,0.000137,4,0.289548,0.339505,0.227644,0.221550,0.121759
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,0.000131,6.772273,0.424172,0.000131,4,0.247473,0.309660,0.177376,0.164386,0.081970


In [34]:
posts_info = posts_info.drop(['text_TFIDF'], axis=1)

#### Загрузим признаки для постов на сервер

In [23]:
posts_info.to_sql('ryagrig_posts_info_lesson_22', con=engine, if_exists='replace')

23